Mixtures of Logit Swissmetro


Preparing the data

In [13]:
import pandas as pd
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, MonteCarlo, log, bioDraws
from biogeme.tools import TemporaryFile

url = "http://transp-or.epfl.ch/data/swissmetro.dat"

# Read the data into a DataFrame
swissmetro = pd.read_csv(url, sep='\t')


database_swissmetro = db.Database('Swissmetro_Model', swissmetro)
globals().update(database_swissmetro.variables) #transform all columns into variables

swissmetro.columns



Index(['GROUP', 'SURVEY', 'SP', 'ID', 'PURPOSE', 'FIRST', 'TICKET', 'WHO',
       'LUGGAGE', 'AGE', 'MALE', 'INCOME', 'GA', 'ORIGIN', 'DEST', 'TRAIN_AV',
       'CAR_AV', 'SM_AV', 'TRAIN_TT', 'TRAIN_CO', 'TRAIN_HE', 'SM_TT', 'SM_CO',
       'SM_HE', 'SM_SEATS', 'CAR_TT', 'CAR_CO', 'CHOICE'],
      dtype='object')

Defining Model Parameters

In [8]:
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 1)
B_COST = Beta('B_COST', 0, None, None, 0)

Defining Random parameter, with Halton Draw, for Monte-Carlo Simulation

In [9]:
B_TIME = Beta('B_TIME', 0, None, None, 0)
B_TIME_S = Beta('B_TIME_S', 1, None, None, 0)
B_TIME_RND = B_TIME + B_TIME_S * bioDraws('B_TIME_RND', 'HALTON2')

Defining the Model

In [15]:
V1 = ASC_TRAIN + B_TIME_RND * TRAIN_TT + B_COST * TRAIN_CO
V2 = ASC_SM + B_TIME_RND * SM_TT + B_COST * SM_CO
V3 = ASC_CAR + B_TIME_RND * CAR_TT + B_COST * CAR_CO

V = {1: V1, 2: V2, 3: V3}

av = {1: TRAIN_AV, 2: SM_AV, 3: CAR_AV}

Estimating the Model

In [17]:
prob = models.logit(V, av, CHOICE)
logprob = log(MonteCarlo(prob))

USER_NOTES = (
    'Example of a mixture of logit models with three alternatives, '
    'approximated using Monte-Carlo integration.'
)

the_biogeme = bio.BIOGEME(
    database_swissmetro, logprob, userNotes=USER_NOTES, parameter_file='few_draws.toml'
)
the_biogeme.modelName = 'b05normal_mixture'

File few_draws.toml has been created


BiogemeError: Chosen alternative 0.0 does not appear in availability dict: dict_keys([1, 2, 3])